### The Decoder

同樣地，我們可以有 N 個 Decoder（假設 N=2）。編碼器 (Encoder) 生成的 **向量** 表示是所有解碼器 (Decoder) 的輸入；也就是說，一個解碼器 (Decoder) 接收兩個輸入，一個來自前一個解碼器 (Decoder)，另一個來自編碼器 (Encoder) 生成的 **向量** 表示。

![decoder in a nutshell](https://www.alexisalulema.com/wp-content/uploads/2022/08/encoderNdecoder.png)

為了理解解碼器 (Decoder) 如何生成目標句子，讓我們看看下面的圖片，該圖片將解碼器 (Decoder) 的輸入描述為一個時間序列。

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/encoderNdecoder_time.png)

> \<sos\>: Start Of the Sentence
>
> \<eos\>: End Of the Sentence

在每個時間步驟中，解碼器 (Decoder) 將新生成的單詞與輸入匹配並預測下一個輸入。一旦生成了標記 \<eos\>，解碼器就完成了目標句子的生成。

與編碼器 (Encoder) 的輸入 Embedding 類似，句子 “<sos> El cielo es azul” 必須 Embedding 以供解碼器使用。

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/output.embedding.png)

要了解解碼器 (Decoder) 如何運作，您必須查看下圖中的組件：

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/decoder.block_.png)

#### Masked multi-head attention

掩碼多頭注意力 (Masked multi-head attention) 與多頭注意力 (Multi-head attention) 機制相似，但有一些細微的差別。

在測試期間，只要有前一個值，解碼器就會生成單詞；也就是說，對於輸入“t2: \<sos\> El”，模型僅使用標記 \<sos\> 和 El 進行訓練，注意力機制必須僅匹配到單詞 El，而不是句子中缺失的單詞。序列中的其餘單詞可以被掩碼，這有助於注意力機制在測試期間僅關注可用的單詞。

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/masking_values-1.gif)

我們必須以相同的方式計算注意力矩陣 $\ Z $，不同之處在於句子中被掩碼的單詞對應的值將接收到 $\ -\infty $。例如：

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/QKT_masked.gif)

這樣，我們就可以像對編碼器 (Encoder) 一樣獲得最終的注意力矩陣，並將這個矩陣輸入到下一層的多頭注意力中。

Masked multihead attention $\ = Concatenate( Z_1, Z_2, Z_3, Z_4, Z_5, Z_6, Z_7, Z_8 ) \cdot W_0 $

#### Multi-head Attention

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/decoders.png)

在前面的圖片中，您可以看到解碼器 (Decoder) 的內部細節，每個多頭注意力層接收編碼器 (Encoder) 輸出的表示 $\ R $ 和前一層的掩碼多頭注意力 $\ M $。由於這一層在解碼器和編碼器之間的交互，它也被稱為編碼器-解碼器注意力 (encoder-decoder attention)。

為了計算注意力機制，我們使用矩陣 $\ M $ 獲取查詢矩陣 $\ Q $；並使用矩陣 $\ R $ 獲取鍵矩陣 $\ K $ 和值矩陣 $\ V $。

* $\ Q_i = M × W_i^Q $
* $\ K_i = R × W_i^K $
* $\ V_i = R × W_i^V $ 

$\ Q_i $ 代表從 $\ M $ 獲得的目標句子，而矩陣 $\ K $ 和 $\ V $ 包含從編碼器 (Encoder) 獲得的表示 $\ R $。

以下是獲取自注意力矩陣的過程：

$\ Z_i = \text{Softmax}\left(\frac{Q_iK_i^T}{\sqrt{d_k}} \cdot V_i \right) $

在計算 $\ Q_i \cdot K_i^T $ 的乘積時，我們會發現結果將包含一個近似於單位矩陣的矩陣，這有助於我們理解 $\ Q $（代表目標句子）與 $\ K $（代表源句子）之間的相似程度。例如：

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/attention_matrix_QK.png)

注意力矩陣的獲取方式與前面的部分相同：

Multihead attention $\ = Concatenate( Z_1, Z_2, Z_3, Z_4, Z_5, Z_6, Z_7, Z_8 ) \cdot W_0 $

#### Feedforward Network

解碼器 (Decoder) 中的這一層與編碼器 (Encoder) 中的工作方式相同。**Add and norm** 組件連接輸入和輸出子層，如下圖所示：

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/decoder_with_addNorm.png)

#### Linear and Softmax Layer

解碼器 (Decoder) 學習目標句子的表示，這些表示將被輸入到線性層 (linear) 和 softmax 層。

![](https://www.alexisalulema.com/wp-content/uploads/2022/08/final_blocks.png)

##### Linear Layer

這一層生成的 logits 的大小與詞彙表相同。假設詞彙表為：

Vocabulary = [azul, cielo, El, es]

假設解碼器 (Decoder) 的輸入是 “El”，解碼器生成的 logits 向量，例如將是：

logits = [40, 51, 43, 38]

##### Softmax Layer

當我們將 Softmax 函數應用於上述 logits 向量時，我們會得到一個概率向量，例如：

prob = [0.005, 0.973, 0.015, 0.007]

具有最高概率值的詞彙是“cielo”（天空），因此這個詞是解碼器 (Decoder) 的下一個預測。


## 應該使用哪種 Transformer 架構？

### Transformer: Encoder-Decoder

Transformer 編碼器-解碼器架構用於語言翻譯等任務，模型必須將一種語言的句子輸入並輸出另一種語言的句子。編碼器接收輸入句子並生成其固定大小的向量表示，然後將其輸入解碼器以生成輸出句子。解碼器使用自注意力和交叉注意力 (cross-attention)，注意力機制應用於編碼器的輸出和解碼器的輸入。

最受歡迎的 Transformer 編碼器-解碼器模型之一是 T5（Text-to-Text Transfer Transformer），由 Google 於 2019 年推出。T5 可以針對廣泛的 NLP 任務進行微調，包括語言翻譯、問答、摘要等。

Transformer 編碼器-解碼器架構的實際應用包括 Google 翻譯，它使用 T5 模型在不同語言之間翻譯文本，以及 Facebook 的 M2M-100，一個可以在 100 種不同語言之間進行翻譯的大型多語言機器翻譯模型。

### Transformer: Encoder

Transformer 編碼器架構用於文本分類等任務，模型必須將一段文本分類到幾個預定義類別之一，例如情感分析、主題分類或垃圾郵件檢測。編碼器接收一個標記序列並生成整個序列的固定大小向量表示，然後可以用於分類。

最受歡迎的 Transformer 編碼器模型之一是 BERT（Bidirectional Encoder Representations from Transformers），由 Google 於 2018 年推出。BERT 在大量文本數據上進行預訓練，可以針對廣泛的 NLP 任務進行微調。

與編碼器-解碼器架構不同，Transformer 編碼器只關注輸入序列，不生成任何輸出序列。它對輸入標記應用自注意力機制，使其能夠專注於給定任務中最相關的部分。

### Transformer: Decoder

Transformer 解碼器架構用於語言生成等任務，模型必須根據輸入提示或上下文生成一系列單詞。解碼器接收上下文的固定大小向量表示，並使用它一次生成一個單詞，每個單詞都依賴於之前生成的單詞。

最受歡迎的 Transformer 解碼器模型之一是 GPT-3（Generative Pre-trained Transformer 3），由 OpenAI 於 2020 年推出。GPT-3 是一個大型語言模型，可以生成各種風格和體裁的類人文本。

Transformer 解碼器架構引入了一種稱為三角掩碼的注意力 (triangle masking for attention) 技術，確保注意力機制只關注當前生成的單詞左側的標記。這防止了模型通過查看尚未生成的標記來“作弊”。

Transformer 解碼器架構的實際應用包括文本生成，模型必須根據給定的提示或主題生成故事或文章，以及聊天機器人，模型必須以自然和吸引人的方式生成對用戶輸入的回應。